<a href="https://colab.research.google.com/github/arumshin-dev/python_conda_jupyter/blob/main/codeit/3_5_7_%E1%84%92%E1%85%A1%E1%84%8B%E1%85%B5%E1%84%87%E1%85%B3%E1%84%85%E1%85%B5%E1%84%83%E1%85%B3%E1%84%80%E1%85%A5%E1%86%B7%E1%84%89%E1%85%A2%E1%86%A8_%E1%84%85%E1%85%B5%E1%84%85%E1%85%A2%E1%86%BC%E1%84%8F%E1%85%B5%E1%86%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 하이브리드 검색과 리랭킹



## 하이브리드 검색
키워드 검색(Keyword Search)과 의미 기반 검색(Semantic Search)의 장점을 결합하여 검색 정확도를 높이는 기법

RAG(Retrieval-Augmented Generation) 시스템 구축 시, 단순히 벡터 검색만 사용할 경우 고유명사나 정확한 용어 매칭이 어려울 수 있는데, 하이브리드 서치는 이를 보완(BM25 → 키워드 기반 정확한 검색, Vector Search → 의미 기반 검색)

### 키워드 검색(Sparse Retriever):

- BM25 알고리즘이 대표적: 단어의 빈도와 역문서 빈도를 기반으로 작동하며, 정확한 단어 매칭에 강합니다.

### 의미 검색(Dense Retriever):

- 임베딩(Embedding) 벡터 유사도(Cosine Similarity 등)를 기반: 단어가 달라도 문맥적 의미가 유사한 문서를 찾는 데 강합니다.

In [ ]:
!pip install langchain langchain-community rank_bm25 langchain_openai faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


> LangChain에서는 EnsembleRetriever 클래스를 사용하여 매우 쉽게 하이브리드 서치를 구현할 수 있습니다. 가장 일반적인 조합은 **BM25(키워드)**와 **VectorStore(의미)**의 결합입니다.

In [ ]:
import getpass
import os

# Prompt the user for the OpenAI API key securely
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [ ]:
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

# 1. 샘플 데이터
docs_list = [
    "아이폰 15의 배터리 수명은 20시간입니다.",
    "갤럭시 S24는 AI 기능이 탑재되어 있습니다.",
    "사과는 맛있는 과일입니다.",
    "배터리 절약 모드를 켜면 사용 시간이 늘어납니다."
]

# 2. BM25 retriever
bm25 = BM25Retriever.from_texts(docs_list)
bm25.k = 2

# 3. Vector retriever
embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_texts(docs_list, embedding)
vec = vectorstore.as_retriever(search_kwargs={"k": 2})


# 4. 하이브리드 검색 함수
def hybrid_search(query):
    bm25_docs = bm25.invoke(query)
    vec_docs = vec.invoke(query)

    # 중복 제거 + 점수 재정렬
    doc_dict = {}

    # BM25 결과 → 가중치 0.5
    for doc in bm25_docs:
        doc_dict[doc.page_content] = doc_dict.get(doc.page_content, 0) + 0.5

    # Vector 결과 → 가중치 0.5
    for doc in vec_docs:
        doc_dict[doc.page_content] = doc_dict.get(doc.page_content, 0) + 0.5

    # 점수 높은 순으로 정렬
    ranked_docs = sorted(doc_dict.items(), key=lambda x: x[1], reverse=True)

    # Document 형태로 변환
    return [Document(page_content=content) for content, _ in ranked_docs]


# 5. LCEL Runnable로 감싸기
hybrid_retriever = RunnableLambda(hybrid_search)


In [ ]:
# 6. 검색
query = "스마트폰 배터리 성능"
docs = hybrid_retriever.invoke(query)

for d in docs:
    print("-", d.page_content)


- 배터리 절약 모드를 켜면 사용 시간이 늘어납니다.
- 사과는 맛있는 과일입니다.
- 아이폰 15의 배터리 수명은 20시간입니다.


> BM25 비중을 줄이고 싶으면 수치 ↓
Vector 비중을 높이고 싶으면 수치 ↑

예:

더 의미 검색 중심 → BM25: 0.2, Vector: 0.8

더 키워드 중심 → BM25: 0.6, Vector: 0.4

## 리랭킹(Re-ranking): "속도 vs 정확도"의 타협점

- Retrieval 단계 (Bi-Encoder):

    - 우리가 지금까지 배운 Vector Search나 BM25입니다.
    - 특징: 미리 계산된 벡터를 비교하므로 속도가 엄청 빠릅니다. 하지만 질문과 문서의 디테일한 관계를 완벽히 파악하지는 못합니다. (대략적인 유사도)
    - 비유: 서류 전형 (수천 명의 지원자 중 스펙이 비슷한 50명을 빠르게 추려냄)

### Re-ranking 단계 (Cross-Encoder):

Retrieval이 뽑아온 상위 문서(예: 50개)와 질문을 **하나의 쌍(Pair)**으로 묶어서 AI 모델에 직접 넣습니다.
- 특징: 두 문장의 관계를 깊이 있게 분석하므로 정확도가 매우 높습니다. 하지만 연산량이 많아 속도가 느립니다. (그래서 전체 문서가 아니라 상위 50개만 검사합니다.)
- 비유: 심층 면접 (서류 통과자 50명을 한 명씩 자세히 인터뷰하여 최종 3명을 선발)


- 요약
    - Vector DB로 빠르고 넓게 후보를 찾고(Candidate Generation)
    - Cross-Encoder로 정밀하게 순위를 다시 매깁니다(Re-ranking).

In [ ]:
!pip install langchain-huggingface sentence-transformers rank_bm25

In [ ]:
import getpass
import os

os.environ["HF_TOKEN"] = getpass.getpass("HuggingFace Token: ")

HuggingFace Token: ··········


In [ ]:
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

# ------------------------------
# 1. 문서 준비
# ------------------------------
texts = [
    "강남역에서 가까운 스테이크 맛집 추천 리스트입니다. 분위기 좋은 고기 레스토랑이 많습니다.",
    "서울 강남역에서 파스타가 맛있는 이탈리안 레스토랑 리스트입니다. 크림, 토마토 파스타가 유명합니다.",   # 🎯 정답
    "강남역 맛집 전체 가이드입니다. 파스타부터 고기, 한식까지 다양한 식당을 포함합니다.",
    "서울 홍대 파스타 맛집 추천입니다. 강남역과는 거리가 꽤 있습니다.",
    "부산 남천동 파스타 맛집 리스트입니다. 부산 지역 이탈리안 맛집 모음입니다.",
    "서울 강남역 카페 추천 리스트입니다. 디저트와 커피로 유명한 곳이 많습니다."
]

# ------------------------------
# 2. BM25 Retriever
# ------------------------------
bm25 = BM25Retriever.from_texts(texts)
bm25.k = 3

# ------------------------------
# 3. Vector Retriever
# ------------------------------
embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
vectorstore = FAISS.from_texts(texts, embeddings)
vec = vectorstore.as_retriever(search_kwargs={"k": 3})

# ------------------------------
# 4. 하이브리드 검색 (BM25 + 벡터)
# ------------------------------
def hybrid_search(query: str):
    bm25_docs = bm25.invoke(query)
    vec_docs = vec.invoke(query)

    scores = {}

    # BM25 가중치 0.5
    for d in bm25_docs:
        scores[d.page_content] = scores.get(d.page_content, 0) + 0.5

    # Vector 가중치 0.5
    for d in vec_docs:
        scores[d.page_content] = scores.get(d.page_content, 0) + 0.5

    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return [Document(page_content=c) for c, _ in ranked]

hybrid_retriever = RunnableLambda(hybrid_search)

In [ ]:
# ------------------------------
# 5. Cross-Encoder 로 직접 Re-rank
# ------------------------------
cross_encoder = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base") #문장 비교 특화 모델

def rerank_with_cross_encoder(query: str, docs, top_n: int = 1):
    # (query, 문서) 쌍 만들기
    pairs = [(query, d.page_content) for d in docs]

    scores = list(cross_encoder.score(pairs))   # List[float]

    # 점수 기준 내림차순 정렬
    ranked = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in ranked[:top_n]]

In [ ]:
# ------------------------------
# 6. 리랭킹 테스트
# ------------------------------
query = "서울 강남역 파스타 맛집 추천"
candidate_docs = hybrid_retriever.invoke(query)                         # 1단계: 하이브리드 검색
final_docs = rerank_with_cross_encoder(query, candidate_docs, top_n=1)  # 2단계: re-rank

for d in final_docs:
    print("답변: ", d.page_content)

답변:  서울 강남역에서 파스타가 맛있는 이탈리안 레스토랑 리스트입니다. 크림, 토마토 파스타가 유명합니다.


In [ ]:
# ------------------------------
# 7. 테스트: RAG vs 하이브리드 vs 리랭크 비교
# ------------------------------
query = "서울 강남역 파스타 맛집 추천"

# (1) RAG 기본: 벡터 검색만 사용
vec_docs = vec.invoke(query)

# (2) 하이브리드 검색: BM25 + 벡터
hybrid_docs = hybrid_retriever.invoke(query)

# (3) 하이브리드 + CrossEncoder 리랭크 (최종 답변에 쓸 문서 1개 선택)
reranked_docs = rerank_with_cross_encoder(query, hybrid_docs, top_n=1)

print("=== 1. 벡터 검색만 사용 (RAG 기본) ===")
for i, d in enumerate(vec_docs, 1):
    print(f"{i}. {d.page_content}")

print("\n=== 2. 하이브리드 검색 (BM25 + 벡터) ===")
for i, d in enumerate(hybrid_docs, 1):
    print(f"{i}. {d.page_content}")

print("\n=== 3. 하이브리드 + CrossEncoder 리랭크 (최종 선택 문서) ===")
for d in reranked_docs:
    print("답변에 사용할 문서:", d.page_content)

=== 1. 벡터 검색만 사용 (RAG 기본) ===
1. 서울 강남역에서 파스타가 맛있는 이탈리안 레스토랑 리스트입니다. 크림, 토마토 파스타가 유명합니다.
2. 서울 홍대 파스타 맛집 추천입니다. 강남역과는 거리가 꽤 있습니다.
3. 강남역 맛집 전체 가이드입니다. 파스타부터 고기, 한식까지 다양한 식당을 포함합니다.

=== 2. 하이브리드 검색 (BM25 + 벡터) ===
1. 서울 홍대 파스타 맛집 추천입니다. 강남역과는 거리가 꽤 있습니다.
2. 서울 강남역 카페 추천 리스트입니다. 디저트와 커피로 유명한 곳이 많습니다.
3. 부산 남천동 파스타 맛집 리스트입니다. 부산 지역 이탈리안 맛집 모음입니다.
4. 서울 강남역에서 파스타가 맛있는 이탈리안 레스토랑 리스트입니다. 크림, 토마토 파스타가 유명합니다.
5. 강남역 맛집 전체 가이드입니다. 파스타부터 고기, 한식까지 다양한 식당을 포함합니다.

=== 3. 하이브리드 + CrossEncoder 리랭크 (최종 선택 문서) ===
답변에 사용할 문서: 서울 강남역에서 파스타가 맛있는 이탈리안 레스토랑 리스트입니다. 크림, 토마토 파스타가 유명합니다.
